In [ ]:
#!pip install selectorlib 

In [66]:
from selectorlib import Extractor
import requests 
import json 
from time import sleep
import csv
from dateutil import parser as dateparser
import pandas as pd 

# Create an Extractor by reading from the YAML file
e = Extractor.from_yaml_file('selectors.yml')

url=input("input url: ")
num_reviews= int(input("how many reviews: "))

print (num_reviews)

urls=[]
urls.append(url)
for i in range(2,((num_reviews // 9)-1 )):
    urls.append(url.replace(url[-1],str(i)))

with open("urls.txt",'w') as f:
    for line in urls:
        f.write(line)
        f.write('\n')

def scrape(url):    
    headers = {
        'authority': 'www.amazon.com',
        'pragma': 'no-cache',
        'cache-control': 'no-cache',
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'none',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-dest': 'document',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }

    # Download the page using requests
    print("Downloading %s"%url)
    r = requests.get(url, headers=headers)
    # Simple check to check if page was blocked (Usually 503)
    if r.status_code > 500:
        if "To discuss automated access to Amazon data please contact" in r.text:
            print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
        else:
            print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
        return None
    # Pass the HTML of the page and create 
    return e.extract(r.text)

# product_data = []
with open("urls.txt",'r') as urllist, open('data.csv','w') as outfile:
    writer = csv.DictWriter(outfile, fieldnames=["title","content","date","variant","images","verified","author","rating","product","url"],quoting=csv.QUOTE_ALL)
    writer.writeheader()
    for url in urllist.readlines():
        data = scrape(url) 
        if data:
            for r in data['reviews']:
                r["product"] = data["product_title"]
                r['url'] = url
                if 'verified' in r:
                    if 'Verified Purchase' in r['verified']:
                        r['verified'] = 'Yes'
                    else:
                        r['verified'] = 'Yes'
                r['rating'] = r['rating'].split(' out of')[0]
                date_posted = r['date'].split('on ')[-1]
                if r['images']:
                    r['images'] = "\n".join(r['images'])
                r['date'] = dateparser.parse(date_posted).strftime('%d %b %Y')
                writer.writerow(r)
            # sleep(5)

input url:  https://www.amazon.com/Sony-LinkBuds-Wireless-Canceling-Headphones/product-reviews/B09YL76VSR/ref=cm_cr_getr_d_paging_btm_next_7?ie=UTF8&reviewerType=avp_only_reviews&pageNumber=1
how many reviews:  70


70







In [67]:

df=pd.read_csv(r'data.csv',encoding = 'unicode_escape')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   title     50 non-null     object 
 1   content   50 non-null     object 
 2   date      50 non-null     object 
 3   variant   50 non-null     object 
 4   images    20 non-null     object 
 5   verified  50 non-null     object 
 6   author    50 non-null     object 
 7   rating    50 non-null     float64
 8   product   50 non-null     object 
 9   url       50 non-null     object 
dtypes: float64(1), object(9)
memory usage: 4.0+ KB


In [68]:
df.head()

,title,content,date,variant,images,verified,author,rating,product,url
0,The best true wireless earphones for everyone,"The Sony Linkbuds S are a shrunk-down, improve...",22 May 2022,Color: Black,https://m.media-amazon.com/images/I/61TtFkp9qO...,Yes,Wilfred,5.0,Sony LinkBuds S Truly Wireless Noise Canceling...,https://www.amazon.com/Sony-LinkBuds-Wireless-...
1,Great Alternative (Supplement) to XM4,My Sony XM4's are phenomenal. I really love my...,27 May 2022,Color: Black,NaN,Yes,Riker,5.0,Sony LinkBuds S Truly Wireless Noise Canceling...,https://www.amazon.com/Sony-LinkBuds-Wireless-...
2,"Good at everything, but probably not the best ...","(This is a lengthy review, so in case you are ...",25 May 2022,Color: Black,https://m.media-amazon.com/images/I/612vlIFGwt...,Yes,C,4.0,Sony LinkBuds S Truly Wireless Noise Canceling...,https://www.amazon.com/Sony-LinkBuds-Wireless-...
3,Only had them for a day so far,The noise cancelling is absolutely surreal. Le...,22 May 2022,Color: White,NaN,Yes,Jeremy Baumgart,5.0,Sony LinkBuds S Truly Wireless Noise Canceling...,https://www.amazon.com/Sony-LinkBuds-Wireless-...
4,Great ear buds for a good price,Sony is usually a good brand in my opinion for...,24 May 2022,Color: Black,NaN,Yes,J.B.,5.0,Sony LinkBuds S Truly Wireless Noise Canceling...,https://www.amazon.com/Sony-LinkBuds-Wireless-...
